In [ ]:
import torch 
from torch import nn 
import yfinance as yf
import talib as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly
import pandas as pd 

## Step 1: Getting data 

In [ ]:
#Finance Indicators 
f_indicators = ["MACD", "RSI", "CMO"," MOM", "Bollinger Bands", "SMA"]
ticker= "BTC-USD"
start_date = "2023-01-01"
end_date = "2025-12-31"

data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)
close_price = data["Close"].values.flatten()
data.head(10)
close_price


In [ ]:
data.isna().sum()

In [ ]:
# MACD (TA-Lib returns 3 arrays)
macd, signal, hist = ta.MACD(close_price, fastperiod=12, slowperiod=26, signalperiod=9)

# Add to DataFrame
data["MACD"] = macd
data["Signal"] = signal
data["Hist"] = hist

# RSI
data["RSI"] = ta.RSI(close_price, timeperiod=14)

# CMO
data["CMO"] = ta.CMO(close_price, timeperiod=14)

# MOM
data["MOM"] = ta.MOM(close_price, timeperiod=10)

# Bollinger Bands
upper, middle, lower = ta.BBANDS(close_price, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

# Add to DataFrame
data["BBU"] = upper    # Upper band
data["BBM"] = middle   # Middle band (SMA)
data["BBL"] = lower    # Lower band

# SMA
data["SMA"] = ta.SMA(close_price, timeperiod=20)
data.head(30)
data.isna().sum()

In [ ]:
data.index.values

In [ ]:


# --- Create subplot layout: 3 rows ---
fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    row_heights=[0.5, 0.25, 0.25],
    subplot_titles=[f"{ticker} Price & BB/SMA", "MACD", "RSI"]
)

# --- Row 1: Candlestick + Bollinger Bands + SMA ---
fig.add_trace(
    go.Candlestick(
        x=data.index,
        open=data["Open"].squeeze(),
        high=data["High"].squeeze(),
        low=data["Low"].squeeze(),
        close=data["Close"].squeeze(),
        name="Price"
    ),
    row=1, col=1
)

# Bollinger Bands
fig.add_trace(go.Scatter(
    x=data.index, y=data["BBU"].squeeze(), line=dict(color="red"), name="BB Upper"
), row=1, col=1)
fig.add_trace(go.Scatter(
    x=data.index, y=data["BBM"].squeeze(), line=dict(color="orange"), name="BB Middle"
), row=1, col=1)
fig.add_trace(go.Scatter(
    x=data.index, y=data["BBL"].squeeze(), line=dict(color="green"), name="BB Lower"
), row=1, col=1)

# SMA
fig.add_trace(go.Scatter(
    x=data.index, y=data["SMA"].squeeze(), line=dict(color="blue", dash="dash"), name="SMA"
), row=1, col=1)

# --- Row 2: MACD + Signal ---
fig.add_trace(go.Scatter(
    x=data.index, y=data["MACD"].squeeze(), line=dict(color="blue"), name="MACD"
), row=2, col=1)
fig.add_trace(go.Scatter(
    x=data.index, y=data["Signal"].squeeze(), line=dict(color="red"), name="MACD Signal"
), row=2, col=1)

# --- Row 3: RSI ---
fig.add_trace(go.Scatter(
    x=data.index, y=data["RSI"].squeeze(), line=dict(color="purple"), name="RSI"
), row=3, col=1)

# --- Layout Improvements ---
fig.update_layout(
    title=f"{ticker} Technical Analysis",
    xaxis_rangeslider_visible=False,
    height=900,
    template="plotly_white",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Add horizontal line for RSI overbought/oversold
fig.add_hline(y=70, line=dict(color="red", dash="dash"), row=3, col=1)
fig.add_hline(y=30, line=dict(color="green", dash="dash"), row=3, col=1)

# Show figure
fig.show()
